Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
import pandas as pd
sales_train_db = "https://raw.githubusercontent.com/IYunet/VIM-2022/main/sales_train.csv"
df_sales_train = pd.read_csv(sales_train_db)
# filter = df_sales_train.loc[df_sales_train.item_cnt_day > 0] # стоит ли сортировать? 
filter = df_sales_train
sort_summ = filter.groupby(by=['item_id']).sum()
top = sort_summ.nlargest(10, 'item_cnt_day').reset_index()
top[["item_id", "item_cnt_day"]]

,item_id,item_cnt_day
0,20949,187642.0
1,2808,17245.0
2,3732,16642.0
3,17717,15830.0
4,5822,14515.0
5,3734,11688.0
6,6675,10289.0
7,3731,10099.0
8,1855,10032.0
9,16787,9227.0


In [ ]:
d_2013 = filter.loc[(pd.to_datetime(filter.date).dt.year) == 2013]
d_2014 = filter.loc[(pd.to_datetime(filter.date).dt.year) == 2014]
d_2015 = filter.loc[(pd.to_datetime(filter.date).dt.year) == 2015]
check_id_each_dates = d_2013[d_2013['item_id'].isin(d_2014['item_id']) & d_2013['item_id'].isin(d_2015['item_id'])]
top[top.item_id.isin(check_id_each_dates.item_id)][["item_id", "item_cnt_day" ]]

,item_id,item_cnt_day
0,20949,187642.0
1,2808,17245.0
2,3732,16642.0
3,17717,15830.0
4,5822,14515.0
5,3734,11688.0
6,6675,10289.0
8,1855,10032.0
9,16787,9227.0


# Задание 2

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [ ]:
items_db = "https://raw.githubusercontent.com/IYunet/VIM-2022/main/items.csv"
item_categories_bd = "https://raw.githubusercontent.com/IYunet/VIM-2022/main/item_categories.csv"
df_items = pd.read_csv(items_db)
df_item_categories = pd.read_csv(item_categories_bd)
big_table = filter.join(df_items, on='item_id', lsuffix='_NEW').join(df_item_categories, on='category_id', lsuffix='_NEW')
filter_big_table = big_table.groupby(by=['item_category_id', 'item_category_name']).sum().reset_index()
second_task_min_category = filter_big_table.loc[filter_big_table.item_cnt_day == 1] # категория, где минимум продаж
second_task_min_category[["item_category_id", "item_category_name"]]

,item_category_id,item_category_name
10,10,Game consoles - PS2
51,51,Books - Cognitive literature


In [ ]:
second_task_max_category = filter_big_table.nlargest(1,"item_cnt_day") # категория, где максимум продаж
second_task_max_category[["item_category_id", "item_category_name"]]

,item_category_id,item_category_name
40,40,Cinema - DVD


#доп задание

In [ ]:

# дополнительное задание 8
# Выведите топ-10 категорий по количеству продаж.
filter_big_table_1 = big_table.groupby(by=['item_category_id']).sum().reset_index()
filter_big_table_1.nlargest(10,"item_cnt_day")[["item_category_id","item_cnt_day"]]

,item_category_id,item_cnt_day
40,40,634171.0
30,30,456540.0
55,55,348591.0
19,19,254887.0
37,37,203284.0
71,71,187998.0
28,28,174954.0
23,23,169944.0
20,20,127319.0
65,65,73077.0


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [ ]:
# сортировка по id магазина, потом посчитать сумму проданного товара, сделать срез топ 5
summ_sells_in_shop = filter.groupby(by=['shop_id']).sum().reset_index()
top_5_shops = summ_sells_in_shop.nlargest(5, 'item_cnt_day')["shop_id"].reset_index()

table_min = big_table.loc[big_table.item_category_id.isin(second_task_min_category.item_category_id	) & 
                          big_table.shop_id.isin(top_5_shops.shop_id)].groupby(by=["item_id"]).mean().reset_index()
table_min[["item_id", "item_cnt_day", "item_category_id"]]


# с минимальными категориями задание 3.1
#пустой результат

,item_id,item_cnt_day,item_category_id


In [ ]:
table_max = big_table.loc[big_table.item_category_id.isin(second_task_max_category.item_category_id) &
                          big_table.shop_id.isin(top_5_shops.shop_id)].groupby(by=["item_id"]).mean().reset_index()
table_max[["item_id", "item_cnt_day", "item_category_id"]]
#ответ с максимальными категориями задание 3.1

,item_id,item_cnt_day,item_category_id
0,0,1.000000,40.0
1,2,1.000000,40.0
2,3,1.000000,40.0
3,4,1.000000,40.0
4,5,1.000000,40.0
...,...,...,...
4709,22157,1.000000,40.0
4710,22159,1.142857,40.0
4711,22160,1.000000,40.0
4712,22162,1.481283,40.0


In [ ]:
# отсортировать исходную таблицу по нужным id_items
needed_items = filter.loc[filter.item_id.isin(table_min.item_id) | filter.item_id.isin(table_max.item_id)]

#отсортировать по дате и посчитать сумму продаж
sort_for_date = needed_items.groupby(by=['date','item_id']).sum().reset_index()

# вывести id_item и минимальный месяц 
min_month = sort_for_date.groupby(by=['item_id']).min().reset_index()
month_filter = pd.to_datetime(min_month.date, format='%d.%m.%Y').dt.month
result_1 = min_month.join(month_filter, rsuffix='_month_min')
r1 = result_1[["item_id","date_month_min"]]

#вывести id_item и максимальный месяц 
id_item_max = sort_for_date.groupby(by=['item_id']).max().reset_index()
max_month = pd.to_datetime(id_item_max.date, format='%d.%m.%Y').dt.month
result_2 = id_item_max.join(max_month, rsuffix='_month_max')
r2 = result_2["date_month_max"]
r1.join(r2)

,item_id,date_month_min,date_month_max
0,0,9,9
1,2,11,8
2,3,7,8
3,4,9,9
4,5,12,12
...,...,...,...
4709,22157,6,5
4710,22159,5,5
4711,22160,3,5
4712,22162,3,3


In [ ]:
# 3.1 другая логика для минимуальных месяцев
sort_for_date = needed_items.groupby(by=['date_block_num','item_id']).sum().reset_index()
min_month = sort_for_date.groupby(by=['item_id']).min().reset_index()
min_month

,item_id,date_block_num,shop_id,item_price,item_cnt_day
0,0,20,54,58.0,1.0
1,2,19,54,58.0,1.0
2,3,18,54,58.0,1.0
3,4,20,54,58.0,1.0
4,5,23,54,28.0,1.0
...,...,...,...,...,...
4709,22157,1,25,58.0,1.0
4710,22159,28,82,1197.0,3.0
4711,22160,0,35,28.0,1.0
4712,22162,18,45,28.0,1.0


In [ ]:
# 3.1 другая логика для максимальных месяцев
sort_for_date = needed_items.groupby(by=['date_block_num','item_id']).sum().reset_index()
max_month = sort_for_date.groupby(by=['item_id']).max().reset_index()
max_month

,item_id,date_block_num,shop_id,item_price,item_cnt_day
0,0,20,54,58.00,1.0
1,2,22,54,58.00,1.0
2,3,19,54,100.00,1.0
3,4,20,54,58.00,1.0
4,5,23,54,28.00,1.0
...,...,...,...,...,...
4709,22157,8,56,149.00,1.0
4710,22159,29,377,4389.00,12.0
4711,22160,17,321,1477.00,11.0
4712,22162,33,7359,84936.22,311.0


In [ ]:
# другая логика для 3.2
# отсортировать по дате и посчитать сумму продаж
sort_for_date = needed_items.groupby(by=['date_block_num','item_id']).sum().reset_index()
min_month = sort_for_date.groupby(by=['item_id']).min().reset_index()
min_month[["item_id","date_block_num"]] 
# ответ для минимума

,item_id,date_block_num
0,0,20
1,2,19
2,3,18
3,4,20
4,5,23
...,...,...
4709,22157,1
4710,22159,28
4711,22160,0
4712,22162,18


In [ ]:
# другая логика для 3.2
# отсортировать по дате и посчитать сумму продаж
sort_for_date = needed_items.groupby(by=['date_block_num','item_id']).sum().reset_index()
max_month = sort_for_date.groupby(by=['item_id']).max().reset_index()
max_month[["item_id","date_block_num"]] 
# ответ для максимума

,item_id,date_block_num
0,0,20
1,2,22
2,3,19
3,4,20
4,5,23
...,...,...
4709,22157,8
4710,22159,29
4711,22160,17
4712,22162,33


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
import numpy as np
# сделать сортировку нужных категорий
requre_category = big_table.loc[big_table.item_category_id.isin(second_task_min_category.item_category_id) | 
                                big_table.item_category_id.isin(second_task_max_category.item_category_id)]
copy_requre_category = big_table.loc[big_table.item_category_id.isin(second_task_min_category.item_category_id) | 
                                big_table.item_category_id.isin(second_task_max_category.item_category_id)] # копия таблицы, исходная еще ,скорее всего, понадобится
# столбец с датой поменять на месяцы
months_in_requre = pd.to_datetime(requre_category.date, format='%d.%m.%Y').dt.month
copy_requre_category["date"] = np.where(copy_requre_category.item_price > 0, months_in_requre, copy_requre_category.date)
# отсорировать по месяцам
sort_by_month = copy_requre_category.groupby(by=['date'])

#вывод средней цены для всех годов сразу
mean_by_month = sort_by_month.mean().reset_index() 
mean_by_month[["date", "item_price"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,date,item_price
0,1,251.278070
1,2,255.786903
2,3,260.710658
3,4,264.492528
4,5,257.146561
5,6,255.093226
6,7,250.004207
7,8,251.258147
8,9,255.414391
9,10,258.567297


In [ ]:
#вывод медианной цены
median_by_month = sort_by_month.median().reset_index()
median_by_month[["date", "item_price"]]

,date,item_price
0,1,199.0
1,2,203.0
2,3,225.0
3,4,229.0
4,5,199.0
5,6,199.0
6,7,199.0
7,8,199.0
8,9,199.0
9,10,199.0


In [ ]:
#2013
requre_category_2013 = big_table.loc[big_table.item_category_id.isin(second_task_min_category.item_category_id) | 
                                big_table.item_category_id.isin(second_task_max_category.item_category_id)]
# в 2013 найти месяцы, где средняя цена --->  предыдущий месяц ср цена > текущий месяц ср цена < след месяц ср цена
cond_2013 = requre_category_2013.loc[pd.to_datetime(requre_category_2013.date, format='%d.%m.%Y').dt.year == 2013]
months_in_2013 = pd.to_datetime(cond_2013.date, format='%d.%m.%Y').dt.month 
cond_2013['date'] = np.where(cond_2013.item_price > 0, months_in_2013, cond_2013.date) # нашел в 2013 все месяцы, и по месяцам товары товары
# сортировка по месяцам
sort_2013_by_month = cond_2013.groupby(by="date")
tab_1 = sort_2013_by_month.mean().reset_index()


index_cond = []
for i in range(1,12):
  if( tab_1['item_price'][i-1] > tab_1['item_price'][i] < tab_1['item_price'][i+1]):
    index_cond += [i-1]
tab_1[['date', 'item_price']].loc[tab_1['date'][index_cond]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,date,item_price
7,8,234.470179


In [ ]:
# 2014
requre_category_2014 = big_table.loc[big_table.item_category_id.isin(second_task_min_category.item_category_id) | 
                                big_table.item_category_id.isin(second_task_max_category.item_category_id)]

# в 2014 найти месяцы, где средняя цена --->  предыдущий месяц ср цена > текущий месяц ср цена < след месяц ср цена
cond_2014 = requre_category_2014.loc[pd.to_datetime(requre_category_2014.date, format='%d.%m.%Y').dt.year == 2014]
months_in_2014 = pd.to_datetime(cond_2014.date, format='%d.%m.%Y').dt.month 
cond_2014['date'] = np.where(cond_2014.item_price > 0, months_in_2014, cond_2014.date) # нашел в 2014 все месяцы, и по месяцам товары товары
# сортировка оп месяцам
sort_2014_by_month = cond_2014.groupby(by="date")
tab_2 = sort_2014_by_month.mean().reset_index()

index_cond = []
for i in range(1,11):
  if( tab_2['item_price'][i-1] > tab_2['item_price'][i] < tab_2['item_price'][i+1]):
    index_cond += [i-1]
tab_2[['date', 'item_price']].loc[tab_2['date'][index_cond]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,item_price
4,5,254.364473
9,10,263.751745


In [ ]:
# 2015
requre_category_2015 = big_table.loc[big_table.item_category_id.isin(second_task_min_category.item_category_id) | 
                                big_table.item_category_id.isin(second_task_max_category.item_category_id)]

# в 2015 найти месяцы, где средняя цена --->  предыдущий месяц ср цена > текущий месяц ср цена < след месяц ср цена
cond_2015 = requre_category_2015.loc[pd.to_datetime(requre_category_2015.date, format='%d.%m.%Y').dt.year == 2015]
months_in_2015 = pd.to_datetime(cond_2015.date, format='%d.%m.%Y').dt.month 
cond_2015['date'] = np.where(cond_2015.item_price > 0, months_in_2015, cond_2015.date) # нашел в 2015 все месяцы, и по месяцам товары товары
# сортировка оп месяцам
sort_2015_by_month = cond_2015.groupby(by="date")
tab_3 = sort_2015_by_month.mean().reset_index()

index_cond = []
for i in range(1,10):
  if( tab_3['item_price'][i-1] > tab_3['item_price'][i] < tab_3['item_price'][i+1]):
    index_cond += [i-1]
tab_3[['date', 'item_price']].loc[tab_3['date'][index_cond]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,item_price
1,2,256.362387
7,8,273.242103


In [ ]:
# другая логика для 4.2
# сортировка по нужным месяца
sort_month_2013 = requre_category.loc[requre_category.date_block_num <= 11]
sort_month_2014 = requre_category.loc[requre_category.date_block_num <= 23].loc[requre_category.date_block_num >= 12]
sort_month_2015 = requre_category.loc[requre_category.date_block_num <= 33].loc[requre_category.date_block_num >= 24]
# определение средней цены
mean_price_2013 = sort_month_2013.groupby(by=['date_block_num']).mean().reset_index()
# в 2013 найти месяцы, где средняя цена --->  предыдущий месяц ср цена > текущий месяц ср цена < след месяц ср цена 
tab_1 = mean_price_2013

index_cond = []
for i in range(1,12):
  if( tab_1['item_price'][i-1] > tab_1['item_price'][i] < tab_1['item_price'][i+1]):
    index_cond += [i]
tab_1["date_block_num"][index_cond]
# 2013

7    7
Name: date_block_num, dtype: int64

In [ ]:
mean_price_2014 = sort_month_2014.groupby(by=['date_block_num']).mean().reset_index()
# в 2014 найти месяцы, где средняя цена --->  предыдущий месяц ср цена > текущий месяц ср цена < след месяц ср цена 
tab_1 = mean_price_2014
index_cond = []
for i in range(1,11):
  if( tab_1['item_price'][i-1] > tab_1['item_price'][i] < tab_1['item_price'][i+1]):
    index_cond += [i]
tab_1["date_block_num"][index_cond]
# 2014

4    16
9    21
Name: date_block_num, dtype: int64

In [ ]:
mean_price_2015 = sort_month_2015.groupby(by=['date_block_num']).mean().reset_index()
# в 2015 найти месяцы, где средняя цена --->  предыдущий месяц ср цена > текущий месяц ср цена < след месяц ср цена 
tab_1 = mean_price_2015
index_cond = []
for i in range(1,10):
  if( tab_1['item_price'][i-1] > tab_1['item_price'][i] < tab_1['item_price'][i+1]):
    index_cond += [i]
tab_1["date_block_num"][index_cond]
# 2015

1    25
7    31
Name: date_block_num, dtype: int64